In [29]:
import numpy as np


In [30]:
sigma = np.float64(5.67*10**(-8))
earth_radius = np.float64(6371000)
earth_mass = np.float64(5.976 * 10**24)
G = np.float64(6.67 * 10**(-11))
albedo_coef = 0.3
earth_radiation = 240

As = np.array([0.1,0.1,0.1,0.5,0.5,0.9]) 
e = np.array([0.1,0.1,0.1,0.9,0.9,0.9]) 
qs = np.float64(1367)
generationHeat = np.array([50] * 6)

width = 0.005
Lx = np.float64(1)
Ly = np.float64(1)
Lz = np.float64(1)
R_value = 2

c = 800
p = 2700

height_above_Earth = np.float64(500)*1000
labels = ["Зенит", "Надир", "Перпенд 1", "Перпенд 2", "Теневая", "Солнечная"]


In [31]:
radius = height_above_Earth + earth_radius # в метры
velocity = np.sqrt(G * earth_mass / radius) # скорость 
length = 2*np.pi*radius #длина орбиты
period = length/velocity #период полного оборота

angleSunSet = np.arccos(earth_radius/(earth_radius+height_above_Earth))
tDay = period * (np.pi/2+angleSunSet)/np.pi

angleSunSet, tDay, period


(0.38384819529001624, 3526.626721270226, 5668.153198351584)

In [32]:
areas = np.array([Ly*Lz, Ly*Lz,
                Lx*Lz, Lx*Lz,
                Lx*Ly, Lx*Ly])

areas


array([1., 1., 1., 1., 1., 1.])

In [33]:
mass = p*width*areas[:]
mass


array([13.5, 13.5, 13.5, 13.5, 13.5, 13.5])

In [34]:
Qav = np.zeros(6)
Qav += generationHeat
Qav

# От солнца
Qav[0]+=areas[0]*As[0]*qs/np.pi #зенит
Qav[1]+=areas[1]*As[1]*(1-np.cos(angleSunSet))*qs/np.pi #надир после 6 часов
Qav[5]+=areas[5]*As[5]*(1+np.sin(angleSunSet))*qs/np.pi #после до 6 часов

#От солнечного альбедо
Qav[1]+=As[1]*areas[1]*albedo_coef*qs/np.pi

#От земли
Qav[1]+=e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2
for i in range(6):
    print(f"{labels[i]}:{str(Qav[i])}")


Зенит:93.5129614413242
Надир:86.85445837237475
Перпенд 1:50.0
Перпенд 2:50.0
Теневая:50.0
Солнечная:588.2737204573177


In [35]:
R = R_value * np.ones(shape=(6,6))
for i in range(6):
    R[i][i] = 10**46
for i in range(3):
    R[2*i+1][2*i] = 10**46
    R[2*i][2*i+1] = 10**46
R


array([[1.e+46, 1.e+46, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [1.e+46, 1.e+46, 2.e+00, 2.e+00, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+46, 1.e+46, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 1.e+46, 1.e+46, 2.e+00, 2.e+00],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+46, 1.e+46],
       [2.e+00, 2.e+00, 2.e+00, 2.e+00, 1.e+46, 1.e+46]])

In [36]:
Tav = 300*np.ones(6)

eps = 0.001
eps_curr = 100
iter = 0

alpha = 0.2

while eps < eps_curr:
    eps_curr = -1
    iter += 1
    for i in range(6):

        newT = (1.0-alpha)*Tav[i] + alpha*(Qav[i] + np.sum(Tav/R[i])) / (sigma*Tav[i]**3*areas[i]*e[i]+np.sum(1/R[i]))
        eps_curr = max(np.abs(newT - Tav[i]), eps_curr)
        Tav[i] = newT

print(f"Total: {iter} iteration")

for i in range(len(labels)):
    print(f"{labels[i]} средняя темпераутра: {Tav[i]:5.2f}K")


Total: 61 iteration
Зенит средняя темпераутра: 292.39K
Надир средняя темпераутра: 289.79K
Перпенд 1 средняя темпераутра: 287.53K
Перпенд 2 средняя темпераутра: 232.43K
Теневая средняя темпераутра: 229.61K
Солнечная средняя темпераутра: 315.82K


In [37]:
Qex = np.zeros(6)
Qex[0] += qs * As[0] * areas[0] * np.sqrt(2)/2
Qex[5] += qs * As[1] * areas[1] * np.sqrt(2.5)/2
Qex[1] += e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2
Qex[1] += 2*albedo_coef*qs*As[1]*areas[1]/np.pi
Qex += generationHeat
Qex


array([146.66149699,  96.74192551,  50.        ,  50.        ,
        50.        , 158.07083904])

In [38]:
Qex - Qav


array([  53.14853555,    9.88746714,    0.        ,    0.        ,
          0.        , -430.20288142])

In [39]:
tDayCalculation = tDay/2


In [40]:
m = np.zeros(R.shape)
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index])) + c*mass[i]/tDayCalculation
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tDayCalculation


In [41]:
tDayCalculation


1763.313360635113

In [42]:
m


array([[ 8.12483308e+00, -1.00000000e-46, -5.00000000e-01,
        -5.00000000e-01, -5.00000000e-01, -5.00000000e-01],
       [-1.00000000e-46,  8.12483308e+00, -5.00000000e-01,
        -5.00000000e-01, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01,  8.12483308e+00,
        -1.00000000e-46, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01, -1.00000000e-46,
         8.12483308e+00, -5.00000000e-01, -5.00000000e-01],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
        -5.00000000e-01,  8.12483308e+00, -1.00000000e-46],
       [-5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
        -5.00000000e-01, -1.00000000e-46,  8.12483308e+00]])

In [43]:
d


array([1843.9940605 , 1784.79897646, 1761.08297208, 1423.62077949,
       1406.35112499, 1504.16800004])

In [44]:
from scipy import linalg
Tmax = linalg.solve(m, d)

Tmax


array([289.47989928, 282.19420079, 282.27027623, 240.73561462,
       240.45923177, 252.49847911])

In [45]:
Qex = generationHeat
Qex[1]+=e[1]*areas[1]*earth_radiation*earth_radius**2/radius**2

Qex


array([50, 70, 50, 50, 50, 50])

In [46]:
Qex - Qav


array([ -43.51296144,  -16.85445837,    0.        ,    0.        ,
          0.        , -538.27372046])

In [47]:
tNight = (period - tDay)/2

m = np.zeros(R.shape)
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index]/2)) + c*mass[i]/tNight
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]/2

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tNight


In [48]:
from scipy import linalg
Tmin = linalg.solve(m, d)

Tmin


array([285.05027339, 285.08803697, 285.65804885, 235.53055654,
       233.51286336, 263.39304646])

In [49]:
labels_temp = ["avg", "min", "max"]
Temps = [Tav, Tmin, Tmax]
for i in range(len(labels)):
    print(f"{labels[i].capitalize()}")
    result = ""
    for j in range(len(Temps)):
        result +=f"{labels_temp[j]}:{Temps[j][i]:5.2f}K "
    print(result)


Зенит
avg:292.39K min:285.05K max:289.48K 
Надир
avg:289.79K min:285.09K max:282.19K 
Перпенд 1
avg:287.53K min:285.66K max:282.27K 
Перпенд 2
avg:232.43K min:235.53K max:240.74K 
Теневая
avg:229.61K min:233.51K max:240.46K 
Солнечная
avg:315.82K min:263.39K max:252.50K 


In [22]:
heat_capacity=mass*c


In [23]:
heat_capacity


array([10800., 10800., 10800., 10800., 10800., 10800.])

In [24]:
for i in range(6):
    for j in range(6):
        if i == j:
            index = R[i] > 0
            m[i][i] = np.abs(np.sum(1/R[i][index]/2)) + heat_capacity[i]/tNight
        else:
            if R[i][j] != 0:
                m[i][j] = -1/R[i][j]/2


In [25]:
%%timeit

for i in range(6):
    for j in range(6):
        if i == j:
            m[i][i] = np.abs(np.sum(1/R[i])) + c*mass[i]/tDayCalculation
        else:
            m[i][j] = -1/R[i][j]

d = Qex[:] - Qav[:] + Tav[:]*c*mass[:]/tDayCalculation


80.3 µs ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
m[np.diag_indices_from(m)]


array([8.12483308, 8.12483308, 8.12483308, 8.12483308, 8.12483308,
       8.12483308])

In [27]:
np.sum(1/R, axis=1) + c*mass[:]/tDayCalculation


array([8.12483308, 8.12483308, 8.12483308, 8.12483308, 8.12483308,
       8.12483308])

In [28]:
%%timeit

m = -1/R
m[np.diag_indices_from(m)] = np.sum(1/R, axis=1) + c*mass[:]/tDayCalculation


31.1 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [51]:
import pandas as pd


In [52]:
a = pd.DataFrame([1, 2, 3, 4, 5, 6])


In [53]:
a.columns = ['a']


In [54]:
a


,a
0,1
1,2
2,3
3,4
4,5
5,6


In [56]:
a['b'] = a['a'] + 2
a['c'] = a['a'] + 2


In [58]:
a['min'] = a[['a'] + ['b']].max(axis=1)
a


,a,b,c,min
0,1,3,3,3
1,2,4,4,4
2,3,5,5,5
3,4,6,6,6
4,5,7,7,7
5,6,8,8,8
